In [16]:
# [1] Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
import os


In [17]:
## [2] Função para carregar a série histórica a partir do CSV
def get_series_from_csv(path='data/btc_limpo.csv'):
    df = pd.read_csv(path)

    if not {'Date', 'Close'}.issubset(df.columns):
        raise RuntimeError('CSV precisa conter colunas "Date" e "Close".')

    df['Date'] = pd.to_datetime(df['Date'])
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df = df.dropna(subset=['Close']).sort_values('Date').reset_index(drop=True)

    # Série diária com preenchimento forward
    s = df.set_index('Date')['Close'].asfreq('D').ffill()
    return s

# Exemplo de carregamento
series = get_series_from_csv('data/btc_limpo.csv')
print('Série carregada:', series.index.min(), 'até', series.index.max(), '| pontos =', len(series))
series.tail()


Série carregada: 2020-06-01 00:00:00 até 2025-05-31 00:00:00 | pontos = 1826


Date
2025-05-27    108994.640625
2025-05-28    107802.328125
2025-05-29    105641.757812
2025-05-30    103998.570312
2025-05-31    104638.093750
Freq: D, Name: Close, dtype: float64

In [18]:
# [3] Função para preparar os dados (treino e teste)
def prepare_data(series, split_ratio=0.8):
    n = len(series)
    split = int(n * split_ratio)
    train = series.iloc[:split]
    test = series.iloc[split:]
    return train, test

train, test = prepare_data(series)
print('Treino:', train.index.min(), 'até', train.index.max(), '| pontos =', len(train))
print('Teste :', test.index.min(), 'até', test.index.max(), '| pontos =', len(test))


Treino: 2020-06-01 00:00:00 até 2024-05-30 00:00:00 | pontos = 1460
Teste : 2024-05-31 00:00:00 até 2025-05-31 00:00:00 | pontos = 366


In [19]:
# [4] Função Prophet
def run_prophet(train, test, horizon):
    df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
    model = Prophet()
    model.fit(df_train)

    future = model.make_future_dataframe(periods=horizon)
    forecast = model.predict(future)

    preds = forecast.set_index('ds')['yhat'].iloc[-horizon:]
    return preds.values

# Exemplo rápido: previsão de 3 dias
prophet_preds = run_prophet(train, test, 3)
print('Prophet (3 dias):', prophet_preds)


01:57:05 - cmdstanpy - INFO - Chain [1] start processing
01:57:06 - cmdstanpy - INFO - Chain [1] done processing


Prophet (3 dias): [60995.68774736 60854.50977639 60777.78182861]


In [20]:
# [5] Função para criar dataset em janelas (para LSTM)
def create_dataset(series, look_back=10):
    X, y = [], []
    for i in range(len(series) - look_back):
        X.append(series[i:(i + look_back)])
        y.append(series[i + look_back])
    return np.array(X), np.array(y)


In [21]:
# [6] Função LSTM
def run_lstm(train, test, horizon, look_back=10, epochs=20):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled = scaler.fit_transform(train.values.reshape(-1,1))

    X_train, y_train = create_dataset(scaled, look_back)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(look_back,1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # usar últimos dados do treino para projetar no futuro
    input_seq = scaled[-look_back:].reshape(1, look_back, 1)
    preds = []
    for _ in range(horizon):
        pred = model.predict(input_seq, verbose=0)[0][0]
        preds.append(pred)
        input_seq = np.append(input_seq[:,1:,:], [[[pred]]], axis=1)

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    return preds

# Exemplo rápido: previsão de 3 dias
lstm_preds = run_lstm(train, test, 3)
print('LSTM (3 dias):', lstm_preds)


LSTM (3 dias): [66356.57  65875.08  65166.633]


In [23]:
# [7] Função de avaliação
def evaluate(y_true, y_pred, label):
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    print(f'{label} - MAPE: {mape:.2f}%')
    return mape


In [24]:
# [8] Rodar previsões para horizontes de 1, 3 e 7 dias
horizons = [1, 3, 7]
results = {}

for h in horizons:
    prophet_preds = run_prophet(train, test, h)
    lstm_preds = run_lstm(train, test, h)

    y_true = test.values[:h]

    mape_prophet = evaluate(y_true, prophet_preds, f'Prophet {h}d')
    mape_lstm = evaluate(y_true, lstm_preds, f'LSTM {h}d')

    results[h] = {
        'Prophet': (prophet_preds, mape_prophet),
        'LSTM': (lstm_preds, mape_lstm),
        'y_true': y_true
    }


01:58:41 - cmdstanpy - INFO - Chain [1] start processing
01:58:42 - cmdstanpy - INFO - Chain [1] done processing
01:58:54 - cmdstanpy - INFO - Chain [1] start processing


Prophet 1d - MAPE: 9.62%
LSTM 1d - MAPE: 0.63%


01:58:54 - cmdstanpy - INFO - Chain [1] done processing
01:59:06 - cmdstanpy - INFO - Chain [1] start processing


Prophet 3d - MAPE: 10.01%
LSTM 3d - MAPE: 2.57%


01:59:07 - cmdstanpy - INFO - Chain [1] done processing


Prophet 7d - MAPE: 12.22%
LSTM 7d - MAPE: 4.26%


In [25]:
# [9] Função para plotar e salvar gráficos
def plot_and_save(y_true, y_pred, label, horizon, model_name):
    plt.figure(figsize=(8,5))
    plt.plot(range(len(y_true)), y_true, label='Real', marker='o')
    plt.plot(range(len(y_pred)), y_pred, label=f'Previsão {model_name}', marker='x')
    plt.title(f'{model_name} - Previsão {horizon} dias')
    plt.legend()
    plt.tight_layout()

    os.makedirs('results', exist_ok=True)
    path = f'results/{model_name}_{horizon}d.png'
    plt.savefig(path)
    plt.close()
    print(f'Gráfico salvo em {path}')

# salvar gráficos
for h in horizons:
    y_true = results[h]['y_true']
    plot_and_save(y_true, results[h]['Prophet'][0], 'Prophet', h, 'Prophet')
    plot_and_save(y_true, results[h]['LSTM'][0], 'LSTM', h, 'LSTM')


Gráfico salvo em results/Prophet_1d.png
Gráfico salvo em results/LSTM_1d.png
Gráfico salvo em results/Prophet_3d.png
Gráfico salvo em results/LSTM_3d.png
Gráfico salvo em results/Prophet_7d.png
Gráfico salvo em results/LSTM_7d.png
